In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import accuracy_score, precision_recall_fscore_support, roc_auc_score, confusion_matrix
from sklearn.preprocessing import label_binarize

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

In [2]:
from art.defences.preprocessor.gaussian_augmentation import GaussianAugmentation

gaussian_augmenter = GaussianAugmentation()

def preprocess(X,y, gaussian_augmenter):
    x_test, y_test = gaussian_augmenter(X, y)

    return torch.tensor(x_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.long)


In [3]:
head = {
            "model" : '',
            "attack_model": '',
            'epsilon': '',
            'Accuracy': '',
            'Macro Precision': '',
            'Weighted Precision': '',
            'Macro Recall': '',
            'Weighted Recall': '',
            'Macro F1': '',
            'Weighted F1': '',

        }
head = pd.DataFrame([head])
head.to_csv("./gaussian_augmenter.csv", mode='a', index=False)




In [4]:
def calculate_performance_metrics(X_test, y_test, model, model_name, attack_name, eps):
    model.eval()
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    
    all_preds = []
    all_labels = []
    probabilities = []

    num_classes = len(np.unique(y_test))
    
    # X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
    X_test_tensor , y_test_tensor= preprocess(X_test, y_test, gaussian_augmenter)
    X_test_tensor = X_test_tensor[:len(X_test)] 
    y_test_tensor = y_test_tensor[:len(y_test)] 
    X_test_tensor = X_test_tensor.to(device)
    y_test_tensor = y_test_tensor.to(device)

    # X_test_tensor = preprocess(X_test, gaussian_augmenter).to(device)
    # y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)
    
    test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
    test_loader = DataLoader(dataset=test_dataset)

    with torch.no_grad():
        
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = torch.argmax(outputs, dim=1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
        all_preds = np.array(all_preds)
        all_labels = np.array(all_labels)
        probabilities = np.array(probabilities)
        
        accuracy = accuracy_score(all_labels, all_preds)

        precision_macro, recall_macro, f1_macro, _ = precision_recall_fscore_support(all_labels, all_preds, average='macro')
        precision_weighted, recall_weighted, f1_weighted, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted')
    
        

        print(f"Accuracy: {accuracy}")
        
        print("\nmacro")
        print(f"Precision: {precision_macro}\nRecall: {recall_macro}\nF1 Score: {f1_macro}")
    
        print("\nweighted")
        print(f"Precision: {precision_weighted}\nRecall: {recall_weighted}\nF1 Score: {f1_weighted}")
        print()
        


        new_row = {
            "model" : model_name,
            "attack_model" : attack_name,
            'epsilon': eps,
            'Accuracy': accuracy,
            'Macro Precision': precision_macro,
            'Weighted Precision': precision_weighted,
            'Macro Recall': recall_macro,
            'Weighted Recall': recall_weighted,
            'Macro F1': f1_macro,
            'Weighted F1': f1_weighted,

        }
        new_row_df = pd.DataFrame([new_row])
        new_row_df.to_csv("./gaussian_augmenter.csv", mode='a', index=False, header=False)




In [5]:
# def calculate_performance_metrics(X_test, y_test, model, model_name, attack_name, eps):
#     model.eval()
#     device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#     model.to(device)
    
#     all_preds = []
#     all_labels = []
#     probabilities = []

#     num_classes = len(np.unique(y_test))
    
#     # X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
#     X_test_tensor , y_test_tensor= preprocess(X_test, y_test, gaussian_augmenter)
#     X_test_tensor = X_test_tensor[:len(X_test)] 
#     y_test_tensor = y_test_tensor[:len(y_test)] 
#     X_test_tensor = X_test_tensor.to(device)
#     y_test_tensor = y_test_tensor.to(device)

#     # X_test_tensor = preprocess(X_test, gaussian_augmenter).to(device)
#     # y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)
    
#     test_dataset = TensorDataset(X_test_tensor, y_test_tensor)
#     test_loader = DataLoader(dataset=test_dataset)

#     with torch.no_grad():
        
#         for inputs, labels in test_loader:
#             inputs, labels = inputs.to(device), labels.to(device)
#             outputs = model(inputs)
#             preds = torch.argmax(outputs, dim=1)
#             all_preds.extend(preds.cpu().numpy())
#             all_labels.extend(labels.cpu().numpy())
#             probabilities.extend(torch.nn.functional.softmax(outputs, dim=1).cpu().numpy())
        
#         all_preds = np.array(all_preds)
#         all_labels = np.array(all_labels)
#         probabilities = np.array(probabilities)

#         np.save(f"/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense_Label/UNSW_Def6/y_pred_{attack_name}{eps}_Def6.npy", all_preds)


In [6]:
x_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_test.npy')
x_train = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_train.npy')
x_val = np.load('/home/jovyan/Sample_Based_Extension/UNSW/x_val.npy')
y_test = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_test.npy')
y_train = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_train.npy')
y_val = np.load('/home/jovyan/Sample_Based_Extension/UNSW/y_val.npy')

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


In [8]:
input_shape = x_train.shape[1]
output_shape = len(np.unique(y_train))

In [9]:
# x_train_tensor = torch.tensor(x_train, dtype=torch.float32).to(device)
x_train_tensor , y_train_tensor= preprocess(x_train, y_train, gaussian_augmenter)
# y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)
x_train_tensor = x_train_tensor.to(device)
y_train_tensor = y_train_tensor.to(device)

# x_val_tensor = torch.tensor(x_val, dtype=torch.float32).to(device)
x_val_tensor , y_val_tensor= preprocess(x_val,y_val, gaussian_augmenter)
# y_val_tensor = torch.tensor(y_val, dtype=torch.long).to(device)
x_val_tensor = x_val_tensor.to(device)
y_val_tensor = y_val_tensor.to(device)


train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=100, shuffle=True)

val_dataset = TensorDataset(x_val_tensor, y_val_tensor)
val_loader = DataLoader(val_dataset, batch_size=100, shuffle=True)

In [10]:
class DNNModel(nn.Module):
    def __init__(self, input_size, output_size):
        super(DNNModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, 30)
        self.fc3 = nn.Linear(30, 20)
        self.fc4 = nn.Linear(20, output_size)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.fc4(x)
        return x

model = DNNModel(input_size=input_shape, output_size=output_shape).to(device)

# Compile model
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_function = nn.CrossEntropyLoss()

# Early stopping variables
min_delta = 0.001
patience = 5
patience_counter = 0
best_loss = float('inf')

In [11]:
# for epoch in range(50):
#     model.train()
#     train_loss = 0.0
#     for inputs, labels in train_loader:
#         inputs, labels = inputs.to(device), labels.to(device)
#         optimizer.zero_grad()
#         outputs = model(inputs)
#         loss = loss_function(outputs, labels)
#         loss.backward()
#         optimizer.step()
#         train_loss += loss.item()

#     avg_train_loss = train_loss / len(train_loader)

#     model.eval()
#     val_train_loss = 0.0
#     correct_predictions = 0
#     with torch.no_grad():
#         for inputs, labels in val_loader:
#             inputs, labels = inputs.to(device), labels.to(device)
#             outputs = model(inputs)
#             loss = loss_function(outputs, labels)
#             val_train_loss += loss.item()
#             _, predicted = torch.max(outputs.data, 1)
#             correct_predictions += (predicted == labels).sum().item()

#     avg_val_loss = val_train_loss / len(val_loader)
#     val_accuracy = correct_predictions / len(val_dataset)

#     print(f"Epoch {epoch+1}, Training Loss: {avg_train_loss:.4f}, Validation Loss: {avg_val_loss:.4f}, Validation Accuracy: {val_accuracy:.4f}")

#     # Early stopping check using min_delta
#     if best_loss - avg_val_loss > min_delta:
#         best_loss = avg_val_loss
#         patience_counter = 0
#     else:
#         patience_counter += 1

#     if patience_counter >= patience:
#         print("Early stopping triggered")
#         break

In [12]:

model.load_state_dict(torch.load("/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense/preprocessor/dnn_pytorch_gaussian_augmenter.pt"))

/tmp/ipykernel_3566/3661704067.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Defense/pr

<All keys matched successfully>

In [13]:
# calculate_performance_metrics(x_test, y_test, model, 'DNN', 'Baseline', '0')

In [14]:
def print_empty_file():
    new_row = {
        "model" : "0",
        "attack_model" : "0",
        'epsilon': "0",
        'Accuracy': "0",
        'Macro Precision': "0",
        'Weighted Precision': "0",
        'Macro Recall': "0",
        'Weighted Recall': "0",
        'Macro F1': "0",
        'Weighted F1': "0",
    }
    new_row_df = pd.DataFrame([new_row])
    new_row_df.to_csv("./gaussian_augmenter.csv", mode='a', index=False, header=False)

In [15]:
# percentage = ["100", "50", "20", "1"]
# model_name = ["XGB", "RF", "DT" ]

percentage = ["10"]
model_name = ["RF"]

In [16]:
epsilon = 0
Def = "Def6"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]



base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}"
                calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()


start RF 10 baseline
(2474, 56) (2474,)
Accuracy: 0.8367016976556184

macro
Precision: 0.8285899858290033
Recall: 0.8600543478260869
F1 Score: 0.8311141811260767

weighted
Precision: 0.8678843820800091
Recall: 0.8367016976556184
F1 Score: 0.840377041861717

start RF 10 BIM
(173650, 56) (173650,)
Accuracy: 0.8071926288511373

macro
Precision: 0.7893617501995884
Recall: 0.8548554407611721
F1 Score: 0.7925948228350432

weighted
Precision: 0.8733972838246031
Recall: 0.8071926288511373
F1 Score: 0.8166799828186181

start RF 10 FGSM
(198139, 56) (198139,)
Accuracy: 0.8263037564538026

macro
Precision: 0.8082050482356766
Recall: 0.848702513429696
F1 Score: 0.8152308252040592

weighted
Precision: 0.8592550810769972
Recall: 0.8263037564538026
F1 Score: 0.8315803203731217

start RF 10 PGD
(173650, 56) (173650,)
Accuracy: 0.8071926288511373

macro
Precision: 0.7893617501995884
Recall: 0.8548554407611721
F1 Score: 0.7925948228350432

weighted
Precision: 0.8733972838246031
Recall: 0.807192628851137

In [17]:
epsilon = 0
Def = "Def6"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]



base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}_ExcludeCaFA/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}_ExcludeCaFA/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}_ExcludeCaFA"
                calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()


start RF 10 baseline
(1208, 56) (1208,)
Accuracy: 0.6216887417218543

macro
Precision: 0.5543191968546017
Recall: 0.5124709332897089
F1 Score: 0.44238964323627655

weighted
Precision: 0.5732078542940182
Recall: 0.6216887417218543
F1 Score: 0.5209147958576975

start RF 10 BIM
(195255, 56) (195255,)
Accuracy: 0.794376584466467

macro
Precision: 0.7829145868091787
Recall: 0.8450988795461626
F1 Score: 0.7814848857375002

weighted
Precision: 0.8673673136149787
Recall: 0.794376584466467
F1 Score: 0.8041871166392655

start RF 10 FGSM
(202801, 56) (202801,)
Accuracy: 0.8166133303090222

macro
Precision: 0.796912768463486
Recall: 0.8430555108676572
F1 Score: 0.8037239334602686

weighted
Precision: 0.8566133620805513
Recall: 0.8166133303090222
F1 Score: 0.8231066398894958

start RF 10 PGD
(195255, 56) (195255,)
Accuracy: 0.794376584466467

macro
Precision: 0.7829145868091787
Recall: 0.8450988795461626
F1 Score: 0.7814848857375002

weighted
Precision: 0.8673673136149787
Recall: 0.794376584466467


In [18]:
epsilon = 0
Def = "Def6"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]



base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"


total_num_attack = ["2", "3"]
for num_attack in total_num_attack:
    for m_name in model_name:
        for p in percentage:
            for attack in attack_names:
                print(f"start {m_name} {p} {attack}")
                
                x_path = f"{base_path}/{m_name}_Exclude{num_attack}Attack/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
                y_path = f"{base_path}/{m_name}_Exclude{num_attack}Attack/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"
    
                try:
                    x_test_adv = np.load(x_path)
                    y_test_adv = np.load(y_path)
                    print(x_test_adv.shape, y_test_adv.shape)
    
                    m_per_name = f"{m_name}{p}_Exclude{num_attack}Attack"
                    calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
                except FileNotFoundError:
                    print(x_path, "not found")
                    print_empty_file()


start RF 10 baseline
(252, 56) (252,)
Accuracy: 0.75

macro
Precision: 0.8724696356275303
Recall: 0.5367647058823529
F1 Score: 0.49540730381718207

weighted
Precision: 0.8137651821862348
Recall: 0.75
F1 Score: 0.6603947493881701

start RF 10 BIM
(177264, 56) (177264,)
Accuracy: 0.8009804585251377

macro
Precision: 0.7841089180599203
Recall: 0.8508777003097074
F1 Score: 0.7860380005087746

weighted
Precision: 0.8714498397641522
Recall: 0.8009804585251377
F1 Score: 0.8110846797351777

start RF 10 FGSM
(187471, 56) (187471,)
Accuracy: 0.8449466850872935

macro
Precision: 0.8282832955554925
Recall: 0.8623375216996253
F1 Score: 0.8358749287318803

weighted
Precision: 0.868794706040361
Recall: 0.8449466850872935
F1 Score: 0.8487305800127141

start RF 10 PGD
(177264, 56) (177264,)
Accuracy: 0.8009804585251377

macro
Precision: 0.7841089180599203
Recall: 0.8508777003097074
F1 Score: 0.7860380005087746

weighted
Precision: 0.8714498397641522
Recall: 0.8009804585251377
F1 Score: 0.81108467973517

/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/opt/conda/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Accuracy: 0.8482200647249191

macro
Precision: 0.5
Recall: 0.42411003236245953
F1 Score: 0.4589388898616705

weighted
Precision: 1.0
Recall: 0.8482200647249191
F1 Score: 0.917877779723341

start RF 10 PGD
(5264, 56) (5264,)
Accuracy: 0.9618161094224924

macro
Precision: 0.5047776130212064
Recall: 0.7310836501901141
F1 Score: 0.5000205555673691

weighted
Precision: 0.9988529600152746
Recall: 0.9618161094224924
F1 Score: 0.9797986601458487

start RF 10 DF
(757, 56) (757,)
Accuracy: 0.47291941875825627

macro
Precision: 0.4642817059483726
Recall: 0.46225072281117946
F1 Score: 0.45979342723004696

weighted
Precision: 0.4941062614994804
Recall: 0.47291941875825627
F1 Score: 0.48014983782040543

start RF 10 AutoPGD
(1416, 56) (1416,)
Accuracy: 0.8637005649717514

macro
Precision: 0.5022144183237525
Recall: 0.5196709585121603
F1 Score: 0.47842516799722123

weighted
Precision: 0.9755074756048138
Recall: 0.8637005649717514
F1 Score: 0.9153027750841989

start RF 10 ZOO
(731, 56) (731,)
Accuracy:

In [15]:
epsilon = 0

Def = "Def6"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

rec_list = ["Euclidean", "cosine", "manhattan"]
for rec in rec_list:
    for attack in attack_names:
        print(f"start {attack}")
        
        x_path = f"{base_path}/Recommendation_{rec}/x_test_adv_{attack}_{Def}.npy"
        y_path = f"{base_path}/Recommendation_{rec}/y_test_adv_{attack}_{Def}.npy"
    
        try:
            x_test_adv = np.load(x_path)
            y_test_adv = np.load(y_path)
            m_per_name = f"Recommendation_{rec}"
            calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
        except FileNotFoundError:
            print(x_path, "not found")
            print_empty_file()



start baseline
Accuracy: 0.9304473304473304

macro
Precision: 0.9332039911308204
Recall: 0.93664563617245
F1 Score: 0.930391649448624

weighted
Precision: 0.939739011911961
Recall: 0.9304473304473304
F1 Score: 0.9305842602502721

start BIM
Accuracy: 0.9059517216666124

macro
Precision: 0.8254578199167844
Recall: 0.8386613153122409
F1 Score: 0.8318240434946158

weighted
Precision: 0.9080077702428391
Recall: 0.9059517216666124
F1 Score: 0.9068865015140217

start FGSM
Accuracy: 0.8354272889170794

macro
Precision: 0.8340650322929191
Recall: 0.8377258660883562
F1 Score: 0.834621655766569

weighted
Precision: 0.8388356099751674
Recall: 0.8354272889170794
F1 Score: 0.8358750193292819

start PGD
Accuracy: 0.9059517216666124

macro
Precision: 0.8254578199167844
Recall: 0.8386613153122409
F1 Score: 0.8318240434946158

weighted
Precision: 0.9080077702428391
Recall: 0.9059517216666124
F1 Score: 0.9068865015140217

start DF
Accuracy: 0.6136676495419026

macro
Precision: 0.5978471028080425
Recall: 

In [ ]:
epsilon = 0
Def = "Def6"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]



base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}_Cluster/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}_Cluster/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}_Cluster"
                calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()


start RF 10 baseline
(16460, 56) (16460,)
Accuracy: 0.9118469015795869

macro
Precision: 0.7224213101279273
Recall: 0.9427206567002618
F1 Score: 0.7818139815069922

weighted
Precision: 0.9587928158947779
Recall: 0.9118469015795869
F1 Score: 0.926183575934014

start RF 10 BIM
(116981, 56) (116981,)


In [ ]:
epsilon = 0
Def = "Def6"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]



base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

for m_name in model_name:
    for p in percentage:
        for attack in attack_names:
            print(f"start {m_name} {p} {attack}")
            
            x_path = f"{base_path}/{m_name}_ActiveLearning/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
            y_path = f"{base_path}/{m_name}_ActiveLearning/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"

            try:
                x_test_adv = np.load(x_path)
                y_test_adv = np.load(y_path)
                print(x_test_adv.shape, y_test_adv.shape)

                m_per_name = f"{m_name}{p}_ActiveLearning"
                calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
            except FileNotFoundError:
                print(x_path, "not found")
                print_empty_file()


In [ ]:
epsilon = 0

Def = "Def6"
attack_names = [
    "baseline", "BIM", "FGSM", "PGD", "DF",
    "AutoPGD", "ZOO", "CaFA", "SINIFGSM", "VNIFGSM"
]

active_learning_name = ["DensityWeighted", "BatchMode"]


base_path = "/home/jovyan/Sample_Based_Extension/UNSW/UNSW_Dynamite_Selection_Data"

for ac_name in active_learning_name:
    for m_name in model_name:
        for p in percentage:
            for attack in attack_names:
                print(f"start {m_name} {p} {attack}")
                
                x_path = f"{base_path}/{m_name}_ActiveLearning_{ac_name}/UNSW_Input{p}/x_test_adv_{attack}_{Def}.npy"
                y_path = f"{base_path}/{m_name}_ActiveLearning_{ac_name}/UNSW_Input{p}/y_test_adv_{attack}_{Def}.npy"
    
                try:
                    x_test_adv = np.load(x_path)
                    y_test_adv = np.load(y_path)
                    m_per_name = f"{m_name}{p}_ActiveLearning_{ac_name}"
                    calculate_performance_metrics(x_test_adv, y_test_adv, model, m_per_name, attack, epsilon)
                except FileNotFoundError:
                    print(x_path, "not found")
                    print_empty_file()



In [14]:

# torch.save(model.state_dict(), "./dnn_pytorch_gaussian_augmenter.pt")

In [16]:
# import time

# epsilon_values = [0.01, 0.1, 0.2, 0.3]

# start_time = time.time()

# for epsilon in epsilon_values:
#     filename = f'/home/jovyan/Sample_Based_Extension/UNSW/transfer_attack/x_test_adv_BIM_eps_{epsilon}.npy'
#     x_test_adv = np.load(filename)

#     calculate_performance_metrics(x_test_adv, y_test, model, 'DNN', 'BIM', epsilon)

# end_time = time.time()
# result = end_time - start_time
# print(f"Execution Time: {result:.6f} seconds")